In [46]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

def scrape_NFL_REF_QB(player_name):

    first_name = player_name.split(' ')[0].lower()
    last_name = player_name.split(' ')[1].lower()
    player_url = f'https://www.sports-reference.com/cfb/players/{first_name}-{last_name}-1.html'
    html_content = requests.get(player_url).text
    
    # if player_name == 'Zach Wilson':
    #     first_name = player_name.split(' ')[0].lower()
    #     last_name = player_name.split(' ')[1].lower()
    #     player_url = f'https://www.sports-reference.com/cfb/players/{first_name}-{last_name}-3.html'
    #     html_content = requests.get(player_url).text
        
    # if player_name == 'Justin Fields' or player_name == 'Jordan Love':
    #     first_name = player_name.split(' ')[0].lower()
    #     last_name = player_name.split(' ')[1].lower()
    #     player_url = f'https://www.sports-reference.com/cfb/players/{first_name}-{last_name}-2.html'
    #     html_content = requests.get(player_url).text
    
    # if player_name == 'Daniel Jones':
    #     first_name = player_name.split(' ')[0].lower()
    #     last_name = player_name.split(' ')[1].lower()
    #     player_url = f'https://www.sports-reference.com/cfb/players/{first_name}-{last_name}-4.html'
    #     html_content = requests.get(player_url).text
    
    # if player_name == 'Josh Allen':
    #     first_name = player_name.split(' ')[0].lower()
    #     last_name = player_name.split(' ')[1].lower()
    #     player_url = f'https://www.sports-reference.com/cfb/players/{first_name}-{last_name}-7.html'
    #     html_content = requests.get(player_url).text

    # if player_name == 'Mitchell Trubisky':
    #     first_name = player_name.split(' ')[0].lower()
    #     first_name = 'Mitch'
    #     last_name = player_name.split(' ')[1].lower()
    #     player_url = 'https://www.sports-reference.com/cfb/players/mitch-trubisky-1.html'
    #     html_content = requests.get(player_url).text

    if len(player_name.split(' ')) > 2:
        first_name = player_name.split(' ')[0].lower()
        last_name = player_name.split(' ')[1].lower()
        suffix = player_name.split(' ')[2].lower()
        player_url = f'https://www.sports-reference.com/cfb/players/{first_name}-{last_name}-{suffix}-1.html'
        html_content = requests.get(player_url).text
    

    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find('table', {'id': 'passing_standard'})
    return table

    if not table:
        for idx in range(2, 6):
            time.sleep(3)
            first_name = player_name.split(' ')[0].lower()
            last_name = player_name.split(' ')[1].lower()
            player_url = f'https://www.sports-reference.com/cfb/players/{first_name}-{last_name}-{idx}.html'
            html_content = requests.get(player_url).text
            
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find('table', {'id': 'passing_standard'})
            
            if not table:
                print(f"Passing stats table not found for {player_name}. Failed on index {idx}")
            else:
                print(f"Player found on index {idx}")
                break
    
    try:
        baseline_headers = []
        headers = [th.getText() for th in table.find_all('th')]
        rows = []
        for tr in table.find_all('tr')[1:]:
            cells = [td.getText() for td in tr.find_all('td')]
            if cells: 
                rows.append(cells)
        
        yr_college = len(rows) - 1

        column_names = [
        'G',        
        'Cmp',      
        'Att',      
        'Cmp%',     
        'Yds',      
        'TD',       
        'TD%',      
        'Int',      
        'Int%',     
        'Y/A',      
        'AY/A',     
        'Y/C',      
        'Y/G',      
        'Rate',
        'seasons',
        'name'
        ]
        
        totals = rows[len(rows) - 2]
        return headers
        totals = [stat for stat in totals if stat.strip()]
        totals.append(yr_college)
        totals.append(player_name)

        if len(totals) == len(column_names):
            final = pd.DataFrame({name: [value] for name, value in zip(column_names, totals)})
        else:
            print("Error: Number of stats does not match number of column names.")

        return final    
    except:
        print(f"FAILED: On {player_name}")

In [60]:
tb = scrape_NFL_REF_QB('Jalen Milroe')

In [61]:
tb.find_all('th')

[<th aria-label="Season" class="poptip center" data-stat="year_id" scope="col">Season</th>,
 <th aria-label="Team" class="poptip sort_default_asc center" data-stat="team_name_abbr" scope="col">Team</th>,
 <th aria-label="Conf" class="poptip sort_default_asc center" data-stat="conf_abbr" data-tip="Conference" scope="col">Conf</th>,
 <th aria-label="Class" class="poptip center" data-stat="class" data-tip="Class" scope="col">Class</th>,
 <th aria-label="Pos" class="poptip center" data-stat="pos" data-tip="Position" scope="col">Pos</th>,
 <th aria-label="G" class="poptip center" data-stat="games" data-tip="&lt;strong&gt;Games played&lt;/strong&gt;&lt;br /&gt;Includes bowl games since 2002" scope="col">G</th>,
 <th aria-label="Cmp" class="poptip center" data-stat="pass_cmp" data-tip="Passes completed" scope="col">Cmp</th>,
 <th aria-label="Att" class="poptip center" data-stat="pass_att" data-tip="Passes attempted" scope="col">Att</th>,
 <th aria-label="Cmp%" class="poptip hide_non_quals cen

In [62]:
headers = [th.getText() for th in tb.find_all('th')]

In [67]:
[i for i in headers if '2' in i]

['2021*', '2022*', '2023*', '2024*']